In [7]:
#imports + file upload
from google.colab import files
import numpy as np
from skimage.color import rgb2gray
import os
import pandas as pd
from collections import Counter
from random import shuffle


#uploaded = files.upload()

In [30]:
#data processing

#next steps: use GAN to debias dataset (MIT 6.S191 lec 4, lab 2)
s_turn = []
m_turn = []
adj_turn = []
no_turn = []
starting_value = 0
continue_loop = True
while continue_loop:
  data_file_name = 'processed-training_data-{}.npy'.format(starting_value)
  if os.path.isfile(data_file_name):
    processed_data = np.load(data_file_name,allow_pickle=True)    
    y = processed_data[0]
    y_ls = y[2]
    x = rgb2gray(processed_data[1])

    if (y_ls > 0.875 or y_ls < 0.125):
      #very sharp turn, but it doesn't warrant its own array
      s_turn.append([x,y])
    elif (y_ls > 0.8 or y_ls < 0.2):
      #sharp turn
      s_turn.append([x,y])
    elif (y_ls > 0.7 or y_ls < 0.3):
      #medium turn
      m_turn.append([x,y])
    elif (y_ls > 0.65 or y_ls < 0.35):
      #light turn, doesn't warrant its own array
      m_turn.append([x,y])
    elif (y_ls > 0.575 or y_ls < 0.425):
      #adjustments
      adj_turn.append([x,y])
    else:
      #no turn
      no_turn.append([x,y])
  else:
    continue_loop = False

balanced_data = s_turn + m_turn + adj_turn + no_turn[0:len(m_turn)+len(s_turn)]

shuffle(balanced_data)

print("done! have {} test cases".format(len(balanced_data)))
#we want agent to make small adjustments, so adj_turn isn't being adjusted
#we don't want agent to just go straight, so less of no_turn is being added

#TODO: flip images

done! have 0 test cases


In [25]:
#should save last 20% of training data as testing data

In [ ]:
#set up alexnet